In [0]:
# imports
from pyspark.sql import functions as F
from pyspark.sql.functions import broadcast

# Load Silver tables
orders = spark.table("orders_clean")
customers = spark.table("customers_clean")
order_items = spark.table("order_items_clean")
sellers = spark.table("sellers_clean")
payments = spark.table("order_payments_clean")



In [0]:
from pyspark.sql.functions import broadcast

# Normal join (shuffle join)
df_shuffle = orders.join(customers, "customer_id")
df_shuffle.explain("formatted")

# Broadcast join (force)
df_broadcast = orders.join(broadcast(customers), "customer_id")
df_broadcast.explain("formatted")


In [0]:
# create salts
buckets = 8
salts = spark.range(buckets).withColumnRenamed("id", "__salt")

order_items_salted = order_items.withColumn("__salt", (F.rand()*buckets).cast("int"))
sellers_salted = sellers.crossJoin(salts)

df_skew_fixed = order_items_salted.join(
    sellers_salted,
    (order_items_salted.seller_id == sellers_salted.seller_id) &
    (order_items_salted.__salt == sellers_salted.__salt)
)

df_skew_fixed.explain("formatted")


In [0]:
daily_state_revenue = spark.read.format("delta").load("/Volumes/spark_olist/gold/delta/daily_sales_state")
daily_state_revenue.explain("formatted")
